In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [3]:
import sys
sys.path.append('/home/gd1279/projects/Rainbow')

In [4]:
from __future__ import division
import argparse
import os
from datetime import datetime
import atari_py
import numpy as np
import torch
import wandb

from agent import Agent
from env import make_env
from memory import ReplayMemory
from masker import ALL_MASKERS
from tqdm import tqdm, trange
import pickle
import bz2

import matplotlib
import matplotlib.pyplot as plt
import cv2
import glob
import copy
import imageio

### Repeating arg-parse args just to make my life easier

In [5]:
parser = argparse.ArgumentParser(description='Rainbow')
parser.add_argument('--id', type=str, default='default', help='Experiment ID')
parser.add_argument('--seed', type=int, default=123, help='Random seed')
parser.add_argument('--disable-cuda', action='store_true', help='Disable CUDA')
parser.add_argument('--game', type=str, default='space_invaders', choices=atari_py.list_games(), help='ATARI game')
parser.add_argument('--T-max', type=int, default=int(50e6), metavar='STEPS', help='Number of training steps (4x number of frames)')
parser.add_argument('--max-episode-length', type=int, default=int(108e3), metavar='LENGTH', help='Max episode length in game frames (0 to disable)')
parser.add_argument('--history-length', type=int, default=4, metavar='T', help='Number of consecutive states processed')
parser.add_argument('--architecture', type=str, default='canonical', choices=['canonical', 'data-efficient'], metavar='ARCH', help='Network architecture')
parser.add_argument('--hidden-size', type=int, default=512, metavar='SIZE', help='Network hidden size')
parser.add_argument('--noisy-std', type=float, default=0.1, metavar='σ', help='Initial standard deviation of noisy linear layers')
parser.add_argument('--atoms', type=int, default=51, metavar='C', help='Discretised size of value distribution')
parser.add_argument('--V-min', type=float, default=-10, metavar='V', help='Minimum of value distribution support')
parser.add_argument('--V-max', type=float, default=10, metavar='V', help='Maximum of value distribution support')
parser.add_argument('--model', type=str, metavar='PARAMS', help='Pretrained model (state dict)')
parser.add_argument('--memory-capacity', type=int, default=int(1e6), metavar='CAPACITY', help='Experience replay memory capacity')
parser.add_argument('--replay-frequency', type=int, default=4, metavar='k', help='Frequency of sampling from memory')
parser.add_argument('--priority-exponent', type=float, default=0.5, metavar='ω', help='Prioritised experience replay exponent (originally denoted α)')
parser.add_argument('--priority-weight', type=float, default=0.4, metavar='β', help='Initial prioritised experience replay importance sampling weight')
parser.add_argument('--multi-step', type=int, default=3, metavar='n', help='Number of steps for multi-step return')
parser.add_argument('--discount', type=float, default=0.99, metavar='γ', help='Discount factor')
parser.add_argument('--target-update', type=int, default=int(8e3), metavar='τ', help='Number of steps after which to update target network')
parser.add_argument('--reward-clip', type=int, default=1, metavar='VALUE', help='Reward clipping (0 to disable)')
parser.add_argument('--learning-rate', type=float, default=0.0000625, metavar='η', help='Learning rate')
parser.add_argument('--adam-eps', type=float, default=1.5e-4, metavar='ε', help='Adam epsilon')
parser.add_argument('--batch-size', type=int, default=32, metavar='SIZE', help='Batch size')
parser.add_argument('--learn-start', type=int, default=int(20e3), metavar='STEPS', help='Number of steps before starting training')
parser.add_argument('--evaluate', action='store_true', help='Evaluate only')
parser.add_argument('--evaluation-interval', type=int, default=100000, metavar='STEPS', help='Number of training steps between evaluations')
parser.add_argument('--evaluation-episodes', type=int, default=10, metavar='N', help='Number of evaluation episodes to average over')
parser.add_argument('--evaluation-size', type=int, default=500, metavar='N', help='Number of transitions to use for validating Q')
parser.add_argument('--render', action='store_true', help='Display screen (testing only)')
parser.add_argument('--enable-cudnn', action='store_true', help='Enable cuDNN (faster but nondeterministic)')

# Custom arguments I added

SCRATCH_FOLDER = r'/misc/vlgscratch4/LakeGroup/guy/'

DEFUALT_WANDB_ENTITY = 'augmented-frostbite'
parser.add_argument('--wandb-entity', default=DEFUALT_WANDB_ENTITY)
DEFAULT_WANDB_PROJECT = 'initial-experiments'
parser.add_argument('--wandb-project', default=DEFAULT_WANDB_PROJECT)
DEFAULT_WANDB_DIR = SCRATCH_FOLDER  # wandb creates its own folder inside
parser.add_argument('--wandb-dir', default=DEFAULT_WANDB_DIR)
parser.add_argument('--wandb-omit-watch', action='store_true')
parser.add_argument('--wandb-resume', action='store_true')
DEFAULT_MEMORY_SAVE_FOLDER = os.path.join(SCRATCH_FOLDER, 'rainbow_memory')
parser.add_argument('--memory-save-folder', default=DEFAULT_MEMORY_SAVE_FOLDER)

_StoreAction(option_strings=['--memory-save-folder'], dest='memory_save_folder', nargs=None, const=None, default='/misc/vlgscratch4/LakeGroup/guy/rainbow_memory', type=None, choices=None, help=None, metavar=None)

In [6]:
args = parser.parse_args([])

np.random.seed(args.seed)
# TODO: why not just fix the torch seed to the same one as np?
# torch.manual_seed(np.random.randint(1, 10000))
torch.manual_seed(args.seed)

if torch.cuda.is_available() and not args.disable_cuda:
  args.device = torch.device('cuda')
  # torch.cuda.manual_seed(np.random.randint(1, 10000))
  torch.cuda.manual_seed(args.seed)
  torch.backends.cudnn.enabled = args.enable_cudnn
else:
  args.device = torch.device('cpu')

args.evaluation_size = 20
args.game = 'frostbite'

### Create the environment

In [7]:
args.add_masks = True
args.maskers = None
env = make_env(args)
env.train()
action_space = env.action_space()

In [8]:
val_mem = ReplayMemory(args, args.evaluation_size)
done = True
for _ in range(args.evaluation_size):
  if done:
    state, done = env.reset(), False

  print(state.shape)
  next_state, _, done = env.step(np.random.randint(0, action_space))
  val_mem.append(state, None, None, done)
  state = next_state


torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])


In [9]:
for state in val_mem:
    print(state.shape)

torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])
torch.Size([36, 84, 84])


In [ ]:
torch.stack?

In [ ]:
args.add_masks = True
args.maskers = None
env = make_env(args)
env.train()
action_space = env.action_space()

In [ ]:
NROWS = 1
NCOLS = args.state_depth
COL_WIDTH = 4
ROW_HEIGHT = 4 
WIDTH_SPACING = 0.5
HEIGHT_SPACING = 0


MIN_FRAME_TO_PLOT = 35

torch_states = []
np_states = []

val_mem = ReplayMemory(args, args.evaluation_size)
done = True
for frame in range(40):
    if done:
        state, done = env.reset(), False

    torch_states.append(state)
    np_state = state.cpu().numpy()
    np_states.append(np_state)
    
    if frame >= MIN_FRAME_TO_PLOT:
        state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
        plt.subplots_adjust(wspace=0.2)

        for ax_index, state_frame in enumerate(np_state[-args.state_depth:]):
            ax = plt.subplot(NROWS, NCOLS, ax_index + 1)
            ax.imshow(state_frame, cmap='gray')
        
        plt.show()
        
    action = np.random.randint(0, action_space)
    next_state, reward, done = env.step(action)
    val_mem.append(state, action, reward, done)
    state = next_state


In [ ]:
idxs, states, actions, returns, next_states, nonterminals, weights = val_mem.sample(10)

In [ ]:
t = torch.tensor([[[1, 2], [0, 3]], [[0, 4], [1, 2]]])
print(t)
values, indices = t.max(0)
print(values)
print(indices)

In [ ]:
t2 = t.unsqueeze(0)

torch.cat((t2, *[]))

In [ ]:
torch.squeeze(torch.gather(t, 0, torch.unsqueeze(indices, 0)))
# t[torch.arange(2), indices]

In [ ]:
def cat_stack(tensors):
    if len(tensors[0].shape) == 2:
        tensors = [torch.unsqueeze(tens, 0) for tens in tensors]
        
    return torch.cat(tensors, 0)

In [ ]:
cat_stack([torch.zeros(5, 84, 84) for _ in range(4)]).shape

In [ ]:
'num_outputs' in args

In [ ]:
x = np.random.randint(0, 10, (2, 5, 4))
y = np.random.randint(0, 10, (2, 5, 4, 3))
print(x)
print('Max:')
print(x.max(0))
max_ind = x.argmax(0)
print('Max indices:')
print(max_ind)
print(max_ind.shape)

In [ ]:
x[max_ind, np.arange(x.shape[1])[:,None], np.arange(x.shape[2])] == x.max(0)

In [ ]:
max_y = y[max_ind, np.arange(x.shape[1])[:,None], np.arange(x.shape[2])]
for i in range(x.shape[1]):
    for j in range(x.shape[2]):
        print(max_y[i, j] == y[max_ind[i, j], i, j])

In [ ]:
np.arange(x.shape[0])[:,None]

In [ ]:
t = (3, 4)
np.zeros((5, *t, 2)).shape

### Create the naive, doctored background frame

In [ ]:
state_0, done = env.reset(), False

In [ ]:
plt.imshow(state_0[3].cpu().numpy(), cmap='Greys')

In [ ]:
s_0_np = state_0[3].cpu().numpy()

In [ ]:
plt.imshow(s_0_np[4:12, :], cmap='Greys')

In [ ]:
doctored_s_0_np = np.copy(s_0_np)
doctored_s_0_np[4:12,4:] = doctored_s_0_np[6,10]
doctored_s_0_np[16:31,4:] = doctored_s_0_np[20,10]
doctored_s_0_np[31:74,4:] = doctored_s_0_np[33,10]
plt.imshow(doctored_s_0_np, cmap='Greys')

## Starting to play around with color-based classifiers

In [ ]:
NROWS = 1
NCOLS = 5
COL_WIDTH = 4
ROW_HEIGHT = 4 
WIDTH_SPACING = 0.5
HEIGHT_SPACING = 0


In [ ]:
MIN_FRAME_TO_PLOT = 20

torch_states = []
np_states = []


val_mem = ReplayMemory(args, args.evaluation_size)
done = True
for frame in range(40):
    
    if done:
        state, done = env.reset(), False

    torch_states.append(state)
    np_state = state.cpu().numpy()
    np_states.append(np_state)
    
    if frame >= MIN_FRAME_TO_PLOT:
        plt.imshow(np_state[3], cmap='gray'); plt.show()
        
    next_state, _, done = env.step(np.random.randint(0, action_space))
    val_mem.append(state, None, None, done)
    state = next_state

In [ ]:
NCOLS = 2
NROWS = 1
INDEX = 20

state = np_states[INDEX][3]

for x in np.unique(state):
    count = np.sum(state == x)
#     print(x, count)
    if count > 100:
        state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
        plt.subplots_adjust(wspace=0.2)
        
        plt.suptitle(f'For x = {x:.4f}, we have {count} pixels')
        
        state_ax = plt.subplot(NROWS, NCOLS, 1)
        state_ax.imshow(state, cmap='Greys')
        
        mask_ax = plt.subplot(NROWS, NCOLS, 2)
        mask_ax.imshow((state == x).astype(np.uint8), cmap='Greys')
        

In [ ]:
plt.imshow(state[35:45, 5:], cmap='Greys', norm=matplotlib.colors.Normalize(0, 1))

In [ ]:
np.unique(state[35:45, 5:])

In [ ]:
for x in np.unique(state[35:45, 5:]):
    count = np.sum(state[35:45, 5:] == x)
    print(x, count)


In [ ]:
raw_screen = np.squeeze(env.ale.getScreenGrayscale())

In [ ]:
plt.imshow(raw_screen, cmap='Greys')

In [ ]:
for x in np.unique(raw_screen):
    count = np.sum(raw_screen == x)
    print(x, count)

In [ ]:
for x in np.unique(raw_screen):
    count = np.sum(raw_screen == x)
#     print(x, count)
    if count > 250:
        state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
        plt.subplots_adjust(wspace=0.2)
        
        plt.suptitle(f'For x = {x:.4f}, we have {count} pixels')
        
        state_ax = plt.subplot(NROWS, NCOLS, 1)
        state_ax.imshow(raw_screen, cmap='Greys')
        
        mask_ax = plt.subplot(NROWS, NCOLS, 2)
        mask_ax.imshow((raw_screen == x).astype(np.uint8), cmap='Greys')

## Starting to mess around with the saved state traces

In [ ]:
def load_saved_states(seed, num=0, name='baseline-rainbow', color=False):
    state_type = color and 'color' or 'gray'
    path_glob = os.path.join('../results', f'{name}-{seed}', 'evaluation/states', f'*{num}-{state_type}.pickle')
    path = glob.glob(path_glob)[0]
    
    with open(path, 'rb') as pickle_file:
        return pickle.load(pickle_file)
                        

In [ ]:
states_300_0_gray = load_saved_states(300)
print(states_300_0_gray.shape)

states_300_0_color = load_saved_states(300, color=True)
states_300_0_color.shape

In [ ]:
frame = states_300_0_gray[100]
tens = torch.tensor(frame, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
small_frame = cv2.resize(frame, (84, 84), interpolation=cv2.INTER_LINEAR)
small_tens = torch.nn.functional.interpolate(tens, size=(84, 84), mode='bilinear')
small_tens.shape
small_tens_np = small_tens.numpy().squeeze()

In [ ]:
np.allclose(small_tens_np, small_frame.astype(np.float32))

In [ ]:
np.max(np.abs(small_tens_np - small_frame.astype(np.float32)))

## Let's print out every 50th state to get some exemplars

In [ ]:
for i in range(0, 2300, 50):
    state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
    plt.subplots_adjust(wspace=0.2)

    plt.suptitle(f'Frame {i}')

    gray_ax = plt.subplot(NROWS, NCOLS, 1)
    gray_ax.imshow(states_300_0_gray[i], cmap='gray')

    color_ax = plt.subplot(NROWS, NCOLS, 2)
    color_ax.imshow(states_300_0_color[i])
    
    plt.show()

## Generate color lists for each semantic type of objects

In [ ]:
player_colors = (110, 116, 129, 148, )
# masking the player won't work by color, but perhaps by color + shape around?
floe_colors = (130, 214)
# 153 also captures the score, so perhaps that needs to be better spatially limited
# 148 also captures some of the player, so perhaps we need to mutually exclude with where we infer the player to be?
# 214 captures both the bear in the black background and the white floes, so another spatial limitation?
bad_animal_colors = (153, # birds
                     148, # crabs
                     111, # bear in white background
                     193, # angry yellow things
                     214, # bear in black background
                    ) 
good_animal_colors = (169, # fish
                     )

# The door is harder to capture straight up by color, as it matches with a black row below the snow
# Maybe this would be trivial in color?

In [ ]:
index = 1050

min_count = 1
max_count = 200

frame_row_min = 8
frame_row_max = -25

frame_col_min = 8
frame_col_max = None

full_frame = states_300_0_gray[index]
frame = full_frame[frame_row_min:frame_row_max, frame_col_min:frame_col_max]

for x in np.unique(frame):
    count = np.sum(frame == x)
#     print(x, count)
    if min_count < count < max_count:
        state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
        plt.subplots_adjust(wspace=0.2)
        
        plt.suptitle(f'For x = {x:.4f}, we have {count} pixels')
        
        state_ax = plt.subplot(NROWS, NCOLS, 1)
        state_ax.imshow(frame, cmap='gray')
        
        mask_ax = plt.subplot(NROWS, NCOLS, 2)
        mask_ax.imshow((frame == x).astype(np.uint8), cmap='gray')

## Same thing but in color

In [ ]:
index = 200

frame_row_min = 8
frame_row_max = -25

frame_col_min = 8
frame_col_max = None

min_count = 2000
max_count = 200000

full_frame = states_300_0_color[index]
frame = full_frame[frame_row_min:frame_row_max, frame_col_min:frame_col_max, :]
# frame = full_frame

flattened = frame.reshape(-1, 3)

for x in np.unique(flattened, axis=0):
    count = np.sum(np.equal(flattened, x.reshape(1, 3)))
#     print(x, count)
    if min_count < count < max_count:
        state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
        plt.subplots_adjust(wspace=0.2)
        
        plt.suptitle(f'For x = {x}, we have {count} pixels')
        
        state_ax = plt.subplot(NROWS, NCOLS, 1)
        state_ax.imshow(frame)
        
        mask_ax = plt.subplot(NROWS, NCOLS, 2)
        mask_ax.imshow((np.all(np.equal(frame, x.reshape(1, 1, 3)), axis=2)).astype(np.uint8), cmap='gray')

In [ ]:
frame_row_min = 8
frame_row_max = -25

frame_col_min = 8
frame_col_max = None

subframe_shape = (177, 152)
full_frame_shape = (210, 160)

player_colors = ((162, 98, 33),
                 (162, 162, 42),
                 (198, 108, 58),
                 (142, 142, 142) # also captures the igloo
                )

unvisited_floe_colors = ((214, 214, 214), # unvisited_floes
                        )
visited_floe_colors = ((84, 138, 210), # visited floes
                      )
land_colors = ((192, 192, 192), # the lighter ground in earlier levels
               (74, 74, 74), # the darker ground in later levels
              ) 

land_row_min = 42
land_row_max = 78
land_row_range = (land_row_min, land_row_max)

bad_animal_colors = ((132, 144, 252), # birds -- also captures the score!
                     (213, 130, 74), # crabs -- no more conflict with the player
                     (210, 210, 64), # angry yellow things
                    ) 

bear_colors = ((111, 111, 111), # bear in white background
               (214, 214, 214), # bear in black background -- same as the unvisited floes
              )

good_animal_colors = ((111, 210, 111), # fish
                     )

animal_subframe_row_min = 70

animal_full_frame_row_min = 78
animal_full_frame_row_max = 185
animal_full_frame_row_range = (animal_full_frame_row_min, animal_full_frame_row_max)

igloo_colors = ((142, 142, 142),
                # isolating the igloo door is harder - its black and orange colors both conflict
                )

igloo_subframe_row_min = 26
igloo_subframe_row_max = 47
igloo_subframe_row_range = (igloo_subframe_row_min, igloo_subframe_row_max)

igloo_subframe_col_min = 104
igloo_subframe_col_max = 136
igloo_subframe_col_range = (igloo_subframe_col_min, igloo_subframe_col_max)

igloo_full_frame_row_min = 35
igloo_full_frame_row_max = 55
igloo_full_frame_row_range = (igloo_full_frame_row_min, igloo_full_frame_row_max)

igloo_full_frame_col_min = 112
igloo_full_frame_col_max = 144
igloo_full_frame_col_range = (igloo_full_frame_col_min, igloo_full_frame_col_max)

# The igloo's black also conflicts with a black line below the ice, so we to do the igloo we would location
# Same with the birds, which conflict with the score
# Player also conflict with igloo, which I guess we could also resolve with location 



## Writing a class to do this sort of filtering

In [ ]:
FULL_FRAME_SHAPE = (210, 160)


class ColorFilterMasker:
    def __init__(self, filter_colors, row_range=(None, None), col_range=(None, None), range_whitelist=False):
        self.filter_colors = np.stack([np.array(x).reshape(1, 1, 3) for x in filter_colors], axis=3)
        self.row_range = row_range
        self.col_range = col_range
        self.range_whitelist = range_whitelist
        
    def __call__(self, frame):
        """
        Assumes the frame is of the form [h, w, c]
        """
        mask = np.any(np.all(np.equal(np.expand_dims(frame, 3), self.filter_colors), axis=2), axis=2).astype(np.uint8)
            
        if self.range_whitelist:
            mask[np.r_[0:self.row_range[0], self.row_range[1]:mask.shape[0]], :] = 0
            mask[:, np.r_[0:self.col_range[0], self.col_range[1]:mask.shape[1]]] = 0

        else:
            mask[self.row_range[0]:self.row_range[1],self.col_range[0]:self.col_range[1]] = 0
                
        return mask

In [ ]:
player_masker = ColorFilterMasker(player_colors, igloo_full_frame_row_range, 
                                  igloo_full_frame_col_range)
unvisited_floe_masker = ColorFilterMasker(unvisited_floe_colors, animal_full_frame_row_range, 
                                          (0, full_frame_shape[1]), range_whitelist=True)
visited_floe_masker = ColorFilterMasker(visited_floe_colors, animal_full_frame_row_range, 
                                        (0, full_frame_shape[1]), range_whitelist=True)
land_masker =  ColorFilterMasker(land_colors, land_row_range, 
                                 (0, full_frame_shape[1]), range_whitelist=True)
bad_animal_masker = ColorFilterMasker(bad_animal_colors, animal_full_frame_row_range, 
                                      (0, full_frame_shape[1]), range_whitelist=True)
good_animal_masker = ColorFilterMasker(good_animal_colors, animal_full_frame_row_range, 
                                       (0, full_frame_shape[1]), range_whitelist=True)
bear_filter = ColorFilterMasker(bear_colors, (0, animal_full_frame_row_min), 
                                (0, full_frame_shape[1]), range_whitelist=True)
igloo_masker = ColorFilterMasker(igloo_colors, igloo_full_frame_row_range, 
                                 igloo_full_frame_col_range, range_whitelist=True)


### Printing out a frame and all of its masks

In [ ]:
NCOLS = 7

for frame_index in range(0, 2300, 50):
    state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
    plt.subplots_adjust(wspace=0.2)
    plt.suptitle(f'Frame {frame_index}')
    
    full_frame = states_300_0_color[frame_index]
    # frame = full_frame[frame_row_min:frame_row_max, frame_col_min:frame_col_max, :]
    frame = full_frame

    color_ax = plt.subplot(NROWS, NCOLS, 1)
    color_ax.imshow(frame)
    
    for ax_index, (masker, name) in enumerate(zip((player_masker, floe_masker, bad_animal_masker, 
                                                   good_animal_masker, bear_filter, igloo_masker),
                                                  ('Player', 'Floes', 'Bad Animals', 
                                                   'Good Animals', 'Bears', 'Igloo'))):
        ax = plt.subplot(NROWS, NCOLS, ax_index + 2)
        ax.set_title(name)
        ax.imshow(masker(frame), cmap='gray')
    
    plt.show()

## Plotting a frame, and all of its masked combined

In [ ]:
NCOLS = 2

for frame_index in range(0, 2300, 50):
    state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
    plt.subplots_adjust(wspace=0.2)
    plt.suptitle(f'Frame {frame_index}')
    
    full_frame = states_300_0_color[frame_index]
    # frame = full_frame[frame_row_min:frame_row_max, frame_col_min:frame_col_max, :]
    frame = full_frame

    color_ax = plt.subplot(NROWS, NCOLS, 1)
    color_ax.imshow(frame)
    
    combined_mask = np.zeros_like(frame)
    
    for ax_index, (masker, color) in enumerate(zip((player_masker, unvisited_floe_masker, 
                                                    visited_floe_masker, land_masker, 
                                                    bad_animal_masker,good_animal_masker, 
                                                    bear_filter, igloo_masker),
                                                   ((255, 128, 0), (224, 224, 224), 
                                                    (124, 185, 232), (103, 103, 103),
                                                    (255, 0, 0), (102, 255, 178), 
                                                    (102, 51, 0), (0, 0, 255) ))
                                             ):
        mask = np.expand_dims(masker(frame), 2)
        combined_mask += mask * np.array(color).astype(np.uint8)
        
    mask_ax = plt.subplot(NROWS, NCOLS, 2)
    mask_ax.imshow(combined_mask)
    
    plt.show()


## Now with resizing back down to (84, 84)

In [ ]:
NCOLS = 7

def resize_func(img, size=(84, 84)):
    return cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)
    

for frame_index in range(0, 2300, 50):
    state_figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
    plt.subplots_adjust(wspace=0.2)
    plt.suptitle(f'Frame {frame_index}')
    
    full_frame = states_300_0_color[frame_index]
    # frame = full_frame[frame_row_min:frame_row_max, frame_col_min:frame_col_max, :]
    frame = full_frame

    color_ax = plt.subplot(NROWS, NCOLS, 1)
    color_ax.imshow(resize_func(frame))
    
    for ax_index, (masker, name) in enumerate(zip((player_masker, floe_masker, bad_animal_masker, 
                                                   good_animal_masker, bear_filter, igloo_masker),
                                                  ('Player', 'Floes', 'Bad Animals', 
                                                   'Good Animals', 'Bears', 'Igloo'))):
        ax = plt.subplot(NROWS, NCOLS, ax_index + 2)
        ax.set_title(name)
        ax.imshow(resize_func(masker(frame).astype(np.uint8)), cmap='gray')
    
    plt.show()

### Saving a gif from these

In [ ]:
combined_masks = []

for frame in states_300_0_color:    
    combined_mask = np.zeros_like(frame)
    
    for ax_index, (masker, color) in enumerate(zip((player_masker, unvisited_floe_masker, 
                                                    visited_floe_masker, land_masker, 
                                                    bad_animal_masker,good_animal_masker, 
                                                    bear_filter, igloo_masker),
                                                   ((255, 128, 0), (224, 224, 224), 
                                                    (124, 185, 232), (103, 103, 103),
                                                    (255, 0, 0), (102, 255, 178), 
                                                    (102, 51, 0), (0, 0, 255) ))
                                             ):
        mask = np.expand_dims(masker(frame), 2)
        combined_mask += mask * np.array(color).astype(np.uint8)
        
    combined_masks.append(resize_func(combined_mask))
    
imageio.mimwrite(f'fully-masked-updated.gif', combined_masks, fps=15)

## Test combining with the optical flow

Particularly, make sure to look at frames 135-138, to see what happens when floes change colors

### Strategy:
* For each mask and direction: 
    * Mask out the flow
    * Use a convolution to to smoothen
    * Remask the smoothened flow
* Do we want to take the sign? Check with and without

In [ ]:
import sys
pyflow_folder = '/home/gd1279/projects/pyflow'
if pyflow_folder not in sys.path: sys.path.append(pyflow_folder)
import pyflow
import pyoptflow

In [ ]:
FARNEBACK_PARAMS = dict(
    pyr_scale=0.5,
    levels=3,
    winsize=5, # 15,
    iterations=7, # 3,
    poly_n=3, # 5,
    poly_sigma=0.8, # 1.2,
    flags=0, # cv2.OPTFLOW_USE_INITIAL_FLOW
)

FARNEBACK_FLOW_TOLERANCE = 1e-5

def resize_func(img, size=(84, 84)):
    return cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)

def flow_farneback(x_0, x_1, **kwargs):
    params = copy.copy(FARNEBACK_PARAMS)
    for key in kwargs:
        params[key] = kwargs[key]
            
    flow = cv2.calcOpticalFlowFarneback(x_0.astype(np.float) / 255, 
                                        x_1.astype(np.float) / 255, 
                                        None, **params)
    return flow[:,:,0], flow[:,:,1]


PYFLOW_PARAMS = dict(
    alpha=0.008, # 0.012,
    ratio=0.75,
    minWidth=5, # 20,
    nOuterFPIterations=7,
    nInnerFPIterations=1,
    nSORIterations=30,
    colType=1,  # 0 or default:RGB, 1:GRAY (but pass gray image with shape (h,w,1))
)

PYFLOW_FLOW_TOLERANCE = 0.75

def flow_pyflow(x_0, x_1, **kwargs):
    params = copy.copy(PYFLOW_PARAMS)
    for key in kwargs:
        params[key] = kwargs[key]
            
    x_flow, y_flow, _ =  pyflow.coarse2fine_flow(np.expand_dims(x_0.astype(np.float) / 255, 2), 
                                                 np.expand_dims(x_1.astype(np.float) / 255, 2), 
                                                 **params)
    return x_flow, y_flow


PYOPTFLOW_PARAMS = dict(
    alpha=0.5, # 1.0
    Niter=100,
)


def flow_pyoptflow(x_0, x_1, **kwargs):
    params = copy.copy(PYOPTFLOW_PARAMS)
    for key in kwargs:
        params[key] = kwargs[key]
            
    return pyoptflow.HornSchunck(x_0.astype(np.float) / 255, x_1.astype(np.float) / 255, **params)


PYOPTFLOW_FLOW_TOLERANCE = 0.1


FARNEBACK_CONFIG = (flow_farneback, 'Farneback', FARNEBACK_FLOW_TOLERANCE)
PYFLOW_CONFIG = (flow_pyflow, 'pyflow', PYFLOW_FLOW_TOLERANCE)
PYOPTFLOW_CONFIG = (flow_pyoptflow, 'pyoptflow', PYOPTFLOW_FLOW_TOLERANCE)


def make_blur_func(kernel_size):
    def blur_denoise(img, mask):
        return np.multiply(cv2.blur(img, kernel_size), mask)
    
    return blur_denoise

denoise = make_blur_func((13, 9))


def flow_mask_combination(mask, flow_x, flow_y, threshold, should_threshold=True, should_sign=False, should_blur=False):
    output_flows = []
    for flow in (flow_x, flow_y):
        flow = np.copy(flow)
        
        if should_threshold:
            flow[np.abs(flow) < threshold] = 0
        
        output = flow * mask
        
        if should_sign:
            output = np.sign(output)
        
        if should_blur:
            output = denoise(output, mask)
        
            
        output_flows.append(output)

    return output_flows

def plot_with_flows(frame_indices, flow_config, maskers, masker_names, add_colorbars=False, should_threshold=True, should_sign=False, should_blur=False):
    NROWS = 1
    NCOLS = 3 + 2 * len(maskers)
    COL_WIDTH = 4
    ROW_HEIGHT = 4 
    WIDTH_SPACING = 0.5
    HEIGHT_SPACING = 0
    
    flow_func, flow_name, flow_threshold = flow_config
    
    for frame_index in frame_indices:
        figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
        plt.subplots_adjust(wspace=0.2)
        plt.suptitle(f'Frame {frame_index}')

        full_color_frame = states_300_0_color[frame_index]
        gray_prev_frame, gray_frame = resize_func(states_300_0_gray[frame_index - 1]), resize_func(states_300_0_gray[frame_index])

        for i, img in enumerate((gray_prev_frame, gray_frame)):
            ax = plt.subplot(NROWS, NCOLS, i + 1)
            ax.imshow(img, cmap='gray')
            ax.set_title(f'Frame {frame_index + i}')

        # Plot subtraction between them
        ax = plt.subplot(NROWS, NCOLS, 3)
        ax.imshow(gray_prev_frame - gray_frame, cmap='gray')
        ax.set_title(f'$\\Delta$')
        
        flow_x, flow_y = flow_func(gray_prev_frame, gray_frame)
        
        for masker_index, (masker, name) in enumerate(zip(maskers, masker_names)):
            mask = resize_func(masker(full_color_frame).astype(np.uint8))
            masked_flow_x, masked_flow_y = flow_mask_combination(mask, flow_x, flow_y, flow_threshold, should_threshold, should_blur, should_sign)
            
            normalizer = matplotlib.colors.Normalize(min(np.min(flow_x), np.min(flow_y)), 
                                                     max(np.max(flow_x), np.max(flow_y)))

            x_ax = plt.subplot(NROWS, NCOLS, 4 + (masker_index * 2))
            x_res = x_ax.imshow(masked_flow_x, cmap='gray', norm=normalizer)
            if add_colorbars: figure.colorbar(x_res, ax=x_ax)
            x_ax.set_title(f'{name} x-flow')

            y_ax = plt.subplot(NROWS, NCOLS, 5 + (masker_index * 2))
            y_res = y_ax.imshow(masked_flow_y, cmap='gray', norm=normalizer)
            if add_colorbars: figure.colorbar(y_res, ax=y_ax)
            y_ax.set_title(f'{name} y-flow')

        plt.show()

In [ ]:
maskers = (player_masker, floe_masker, bad_animal_masker)
masker_names = ('Player', 'Floes', 'Bad Animals')
modified_config = (flow_farneback, 'Farneback', 5e-5)

plot_with_flows(range(130, 140), modified_config, maskers, masker_names, should_sign=True) #, should_blur=True, should_sign=True) #, should_sign=True)

In [ ]:
def plot_frames_step_by_step(frame_indices, flow_config, masker, masker_name, add_colorbars=False, should_threshold=True, should_sign=False, should_blur=False):
    NROWS = 1
    NCOLS = 5 + should_threshold + should_blur + should_sign
    COL_WIDTH = 4
    ROW_HEIGHT = 4 
    WIDTH_SPACING = 0.5
    HEIGHT_SPACING = 0
    
    flow_func, flow_name, flow_threshold = flow_config
    
    for frame_index in frame_indices:
        full_color_frame = states_300_0_color[frame_index]
        gray_prev_frame, gray_frame = resize_func(states_300_0_gray[frame_index - 1]), resize_func(states_300_0_gray[frame_index])
        
        flow_x, flow_y = flow_func(gray_prev_frame, gray_frame)
        mask = resize_func(masker(full_color_frame).astype(np.uint8))
        
        for flow, direction in zip((flow_x, flow_y), ('X', 'Y')):
            figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
            plt.subplots_adjust(wspace=0.2)
            plt.suptitle(f'Frame {frame_index} {direction}')

            for i, img in enumerate((gray_prev_frame, gray_frame)):
                ax = plt.subplot(NROWS, NCOLS, i + 1)
                ax.imshow(img, cmap='gray')
                ax.set_title(f'Frame {frame_index - i + }')

            # Plot subtraction between them
            delta_ax = plt.subplot(NROWS, NCOLS, 3)
            delta_ax.imshow(gray_prev_frame - gray_frame, cmap='gray')
            delta_ax.set_title(f'$\\Delta$')
        
            raw_flow_ax = plt.subplot(NROWS, NCOLS, 4)
            raw_flow_ax.imshow(flow, cmap='gray')
            raw_flow_ax.set_title('Raw flow')
            
            if should_threshold:
                flow[np.abs(flow) < flow_threshold] = 0
                thresh_flow_ax = plt.subplot(NROWS, NCOLS, 5)
                thresh_flow_ax.imshow(flow, cmap='gray')
                thresh_flow_ax.set_title('Thresholded flow')

            output = flow * mask
            masked_flow_ax = plt.subplot(NROWS, NCOLS, 5 + should_threshold)
            masked_flow_ax.imshow(output, cmap='gray')
            masked_flow_ax.set_title('Masked flow')

            if should_sign:
                output = np.sign(output)
                signed_flow_ax = plt.subplot(NROWS, NCOLS, 6 + should_threshold)
                signed_flow_ax.imshow(output, cmap='gray')
                signed_flow_ax.set_title('Signed flow')

            if should_blur:
                output = denoise(output, mask)
                blurred_flow_ax = plt.subplot(NROWS, NCOLS, 6 + should_threshold + should_sign)
                blurred_flow_ax.imshow(output, cmap='gray')
                blurred_flow_ax.set_title('Denoised flow')

            plt.show()

In [ ]:
maskers = (player_masker, floe_masker, bad_animal_masker)
masker_names = ('Player', 'Floes', 'Bad Animals')
modified_config = (flow_farneback, 'Farneback', 5e-5)

plot_frames_step_by_step(range(130, 140), modified_config, floe_masker, 'Floes', should_sign=True, should_blur=True, )

In [ ]:
frame_index = 132
gray_prev_frame, gray_frame = states_300_0_gray[frame_index - 1], states_300_0_gray[frame_index]
plt.imshow(gray_frame - gray_prev_frame, cmap='gray')